# German Credit Dataset
- 대출인지 아닌지를 예측하는 문제
- 데이터를 NB에 맞도록 간단하게 변환합니다.
- Binary 데이터들로 이루어진 대출 사기 데이터들로 부터 대출인지 아닌지 예측해보세요.

In [7]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [8]:
data_url = './fraud.csv'
df = pd.read_csv(data_url, sep=',')
df.head()

,ID,History,CoApplicant,Accommodation,Fraud
0,1,current,none,own,True
1,2,paid,none,own,False
2,3,paid,none,own,False
3,4,paid,guarantor,rent,True
4,5,arrears,none,own,False


In [9]:
# ID열을 삭제해줍니다.
df= df.drop("ID",axis=1)

# Label(Y_data)을 따로 저장해 줍니다.
Y_data = df["Fraud"]
df= df.drop("Fraud",axis=1)

In [10]:
df.head()

,History,CoApplicant,Accommodation
0,current,none,own
1,paid,none,own
2,paid,none,own
3,paid,guarantor,rent
4,arrears,none,own


In [11]:
Y_data

0      True
1     False
2     False
3      True
4     False
5      True
6     False
7     False
8     False
9      True
10    False
11     True
12     True
13    False
14    False
15    False
16    False
17    False
18    False
19    False
Name: Fraud, dtype: bool

In [12]:
# as_matrix()함수를 통해 array형태로 변환시켜 줍니다.
# Convert the frame to its Numpy-array representation.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.as_matrix.html

Y_data = Y_data.as_matrix() # numpy로 바꿈.
Y_data

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


array([ True, False, False,  True, False,  True, False, False, False,
        True, False,  True,  True, False, False, False, False, False,
       False, False])

In [13]:
type(Y_data)

numpy.ndarray

In [14]:
df.head()

,History,CoApplicant,Accommodation
0,current,none,own
1,paid,none,own
2,paid,none,own
3,paid,guarantor,rent
4,arrears,none,own


In [15]:
# 우리가 앞으로 사용할 데이터 셋입니다. 그런데 문제가 있어보이네요...

## One-Hot encoding

* 범주형 변수를 dummy변수로 변환해주는 작업
* Do it yourself!

In [16]:
df['History'].unique()

array(['current', 'paid', 'arrears', 'none'], dtype=object)

In [17]:
df['CoApplicant'].unique()

array(['none', 'guarantor', 'coapplicant'], dtype=object)

In [18]:
dummy_var_history = pd.get_dummies(df['History']) # Initial에 한 변수였던 애들을 여러 column으로 바꿔버리기
dummy_var_history.columns =  ['History_' + str(col) for col in dummy_var_history.columns]
dummy_var_co = pd.get_dummies(df['CoApplicant']) # Initial에 한 변수였던 애들을 여러 column으로 바꿔버리기
dummy_var_co.columns =  ['CoApplican_' + str(col) for col in dummy_var_co.columns]
dummy_var_aco = pd.get_dummies(df['Accommodation']) # Initial에 한 변수였던 애들을 여러 column으로 바꿔버리기
dummy_var_aco.columns =  ['Accommodation_' + str(col) for col in dummy_var_aco.columns]

In [19]:
df = pd.concat([df,dummy_var_history,dummy_var_co,dummy_var_aco],axis=1)

In [20]:
df.head()

,History,CoApplicant,Accommodation,History_arrears,History_current,History_none,History_paid,CoApplican_coapplicant,CoApplican_guarantor,CoApplican_none,Accommodation_free,Accommodation_own,Accommodation_rent
0,current,none,own,0,1,0,0,0,0,1,0,1,0
1,paid,none,own,0,0,0,1,0,0,1,0,1,0
2,paid,none,own,0,0,0,1,0,0,1,0,1,0
3,paid,guarantor,rent,0,0,0,1,0,1,0,0,0,1
4,arrears,none,own,1,0,0,0,0,0,1,0,1,0


### 1. Do One-Hot Encoding! 

In [21]:
# 범주형 변수 처리 문제입니다.
# 앞선 EDA 시간과 Logistic EDA를 통해 우리는 범주형 변수를 처리해 주는 방법을 배웠습니다.
# get_dummies를 사용해서 One-Hot encoding 처리를 해주세요.

x_df = df.drop(['History','CoApplicant','Accommodation'],axis=1)
x_df.head() # dummy변수로 변환

,History_arrears,History_current,History_none,History_paid,CoApplican_coapplicant,CoApplican_guarantor,CoApplican_none,Accommodation_free,Accommodation_own,Accommodation_rent
0,0,1,0,0,0,0,1,0,1,0
1,0,0,0,1,0,0,1,0,1,0
2,0,0,0,1,0,0,1,0,1,0
3,0,0,0,1,0,1,0,0,0,1
4,1,0,0,0,0,0,1,0,1,0


* One-Hot Encoding이 제대로 되었다면 우리는 10개의 Feature를 얻을 수 있습니다.

In [22]:
x_data = x_df.as_matrix()
x_data

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0]], dtype=uint8)

In [23]:
# one-hot encoding을 통해 10개의 Feature를 얻었다.

#### Q1. as_matrix()함수를 통해 우리가 하고자 하는 것은 무엇일까요? 

In [24]:
Y_data == True # boolean index

array([ True, False, False,  True, False,  True, False, False, False,
        True, False,  True,  True, False, False, False, False, False,
       False, False])

In [25]:
len(set(Y_data))

2

## Naive bayes classifier

* P(Y)
* P(X1, X2, ..., Xn)
* P(Y|X1, X2, X3, ..., Xn)
* P(X1|Y), P(X2|Y), ... P(Xn|Y)
등 우리가 구해야 할 식들에 대한 아이디어가 있어야 합니다.

### P(Y1), P(Y0) 구하기

In [26]:
# P(Y1), P(Y0)
# P(Y1) = count(Y1) / count(Y)

P_Y_True = sum(Y_data==True) / len(Y_data)
P_Y_False = 1 - P_Y_True

P_Y_True, P_Y_False

(0.3, 0.7)

* 이번 튜토리얼에서는 **index를 이용합니다.**
* 이해하기보다는 따라 하면서 음미해보세요.

In [27]:
# y가 1일 경우, y가 0일 경우를 구해줘야 합니다.
# 이번 시간에는 np.where를 사용합니다.
# np.where

In [28]:
ix_Y_True = np.where(Y_data) # Y_data == True인 인덱스를 뽑아줍니다.
ix_Y_False = np.where(Y_data==False)

ix_Y_True, ix_Y_False

((array([ 0,  3,  5,  9, 11, 12]),),
 (array([ 1,  2,  4,  6,  7,  8, 10, 13, 14, 15, 16, 17, 18, 19]),))

In [29]:
# np.where을 사용해서 Y가1일 때와 0일 때 각각의 인덱스 값을 얻을 수 있게 되었습니다.

In [30]:
# P(X|Y) = count(X_cap_Y) / count(Y)

### productP(X|Yc) 구하기

* product * P(X|Y1)
* product * P(X|Y2)

### 그냥 곱으로 계산하기

In [31]:
# y가 true일 때 결과값
p_x_y_true = sum(x_data[ix_Y_True])/sum(Y_data==True)

In [32]:
p_x_y_true = (x_data[ix_Y_True].sum(axis=0) / sum(Y_data==True))  # Q.뒤에 sum(Y_data == True) 필요한가요? # 앞에 식이 P(X_cap_Y1)인 것 같은데...
p_x_y_false = (x_data[ix_Y_False].sum(axis=0) / sum(Y_data==False))

p_x_y_true, p_x_y_false

(array([0.16666667, 0.5       , 0.16666667, 0.16666667, 0.        ,
        0.16666667, 0.83333333, 0.        , 0.66666667, 0.33333333]),
 array([0.42857143, 0.28571429, 0.        , 0.28571429, 0.14285714,
        0.        , 0.85714286, 0.07142857, 0.78571429, 0.14285714]))

In [33]:
# p_x_y_true.dot(x_test)은 계산이 "합하는 거라" log 계산하듯이 하는건데 원래 처음 아이디어는 "곱하는" 거였으니까 곱하고 싶습니다!
p_x_y_true

array([0.16666667, 0.5       , 0.16666667, 0.16666667, 0.        ,
       0.16666667, 0.83333333, 0.        , 0.66666667, 0.33333333])

In [36]:
x_test = [0,1,0,0,0,1,0, 0,1,0]

np.array(x_test)==1

array([False,  True, False, False, False,  True, False, False,  True,
       False])

In [37]:
# 1인 것만 곱하기(여기서는 3번 이런건 없으므로 그냥 1인거 곱해주기만 하면 될 듯)
p_x_y_true[np.array(x_test)==1]

array([0.5       , 0.16666667, 0.66666667])

In [38]:
p_y_true_test = np.prod(p_x_y_true[np.array(x_test)==1])
p_y_false_test = np.prod(p_x_y_false[np.array(x_test)==1])

p_y_true_test

0.05555555555555555

In [39]:
p_y_false_test

0.0

In [40]:
p_y_true_test < p_y_false_test

False

# 우리가 배웠던 것처럼 오류가 있는데 smoothing이나 log 처리를 해보세요

In [41]:
# 총 10개의 값에 대해서 확률을 구해준다.

## LOG

In [42]:
x_test

[0, 1, 0, 0, 0, 1, 0, 0, 1, 0]

In [43]:
p_x_y_true

array([0.16666667, 0.5       , 0.16666667, 0.16666667, 0.        ,
       0.16666667, 0.83333333, 0.        , 0.66666667, 0.33333333])

#### 진짜 로그로더해보기

In [44]:
log_p_x_y_true = np.log(p_x_y_true)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [45]:
log_p_x_y_true

array([-1.79175947, -0.69314718, -1.79175947, -1.79175947,        -inf,
       -1.79175947, -0.18232156,        -inf, -0.40546511, -1.09861229])

In [46]:
# log로 계산하려고 하면 0은 무한대가 되니까 좀 문제가 있네..
log_p_x_y_true.dot(x_test)  # 곱한 값을 다 더하는거다.a

nan

-> 안돌아감(log(0)을 함으로써 생기는 문제)

#### 방법1. 입실론 더하기(스무딩을 하는 방법도 있음) -> 값은 나옴

In [47]:
epsilon = 0.00001

In [50]:
log_p_x_y_true = np.log(p_x_y_true + epsilon)

In [51]:
blog_p_x_y_true

array([ -1.79169947,  -0.69312718,  -1.79169947,  -1.79169947,
       -11.51292546,  -1.79169947,  -0.18230956, -11.51292546,
        -0.40545011,  -1.09858229])

In [52]:
log_p_x_y_true.dot(x_test)  # 곱한 값을 다 더하는거다.a

-2.8902767600085895

#### 방법2. 컨셉상으로 로그는 안하고 더하기만 하기

In [161]:
# 이미 여기에서 dot 하면 3을 곱하게 되므로 Chinese가 세번 나왔으면 3제곱 하는 의미이다.
p_x_y_true.dot(x_test)  # 곱한 값을 다 더하는거다.a

1.3333333333333333

In [162]:
x_test = [0,1,0,0,0,1,0, 0,1,0]


import math
# log니까 바로 dot하면 안되고 
# P_Y_True에 더하기 하는 이유는 앞에 제곱이 아니라 곱이기 때문이다.
p_y_true_test = P_Y_True + p_x_y_true.dot(x_test) 
p_y_false_test = P_Y_False + p_x_y_false.dot(x_test) 

p_y_true_test, p_y_false_test

(1.6333333333333333, 1.7714285714285714)

In [165]:
p_y_true_test < p_y_false_test

True

## Smmothing

## 2. Do Smoothing을 통해 P(Y=1|X)의 확률과 P(Y=0|X)의 확률 값을 비교하세요.

In [166]:
smoothing_p = 2


In [171]:
# 전체 class 개수는 2개
p_x_y_true = (x_data[ix_Y_True].sum(axis=0) + smoothing_p) / (sum(Y_data==True)+smoothing_p*2)  # Q.뒤에 sum(Y_data == True) 필요한가요? # 앞에 식이 P(X_cap_Y1)인 것 같은데...
p_x_y_false = (x_data[ix_Y_False].sum(axis=0)+smoothing_p) / (sum(Y_data==False)+smoothing_p*2)

p_x_y_true, p_x_y_false

(array([0.3, 0.5, 0.3, 0.3, 0.2, 0.3, 0.7, 0.2, 0.6, 0.4]),
 array([0.44444444, 0.33333333, 0.11111111, 0.33333333, 0.22222222,
        0.11111111, 0.77777778, 0.16666667, 0.72222222, 0.22222222]))

### -> p_x_y_false값 중에 0이었던 값이 0.11111111이 되었다
array([0.42857143, 0.28571429, 0.        , 0.28571429, 0.14285714,0.        , 0.85714286, 0.07142857, 0.78571429, 0.14285714]))

#### smoothing에 곱으로 계산하기


In [173]:
p_y_true_test = np.prod(p_x_y_true[np.array(x_test)==1])
p_y_false_test = np.prod(p_x_y_false[np.array(x_test)==1])

p_y_true_test, p_y_false_test


(0.09, 0.026748971193415638)

-> 스무딩 안했을 때 곱 계산하면 0.05555555555555555 0 인데 (0.09, 0.026748971193415638)로 확률이 어느정도 늘어남

In [174]:
p_y_true_test < p_y_false_test

False

#### log 컨셉으로 해보기

In [169]:
import math
x_test = [0,1,0,0,0,1,0, 0,1,0]


p_y_true_test = P_Y_True+p_x_y_true.dot(x_test)
p_y_false_test = P_Y_False+p_x_y_false.dot(x_test)

p_y_true_test, p_y_false_test

(1.7, 1.8666666666666665)

-> 스무딩 했을때 로그 계산값도 (1.6333333333333333, 1.7714285714285714) 에서 (1.7, 1.8666666666666665)로 전반적으로 는다

In [170]:
p_y_true_test < p_y_false_test

True

# 결과값에 대한 설명과 해석을 달아주세요

스무딩을 하면 전반적으로 확률값이 높아집니다!
그리고 실제로 곱을 하는것보다 로그 컨셉으로 계산하는 방법이 값 자체가 커서 비교하기에 좋습니다(0으로 줄어들지 않음)